In [10]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import collections
import sklearn.model_selection

from classes import *
from files import create_outfile_each_fold, create_outfile_mean, get_path_outfile_each_fold, get_path_mean, save_figs
from gpu import set_avx_avx2, set_gpu
from metrics import calculate_iou_dice, dice_loss, jaccard_distance_loss
from model import cfg_model, train_model

In [11]:
tensorflow.__version__

'2.8.2'

# GPU

In [12]:
set_avx_avx2()
set_gpu()

GPU: /physical_device:GPU:0


In [31]:
cfg = {
    "batch_size": 8,
    "cross_validation": 2,
    "epochs": 1,
    "image_size": 400,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_base": "dataset",
    "path_out": "out"
}

In [21]:
def get_index(cfg):
    return sklearn.model_selection.StratifiedShuffleSplit(n_splits=cfg["cross_validation"], test_size=cfg["test_size"], random_state=cfg["random_state"])

def get_list_index(cfg, x, y):
    list_index = get_index(cfg)
    return list([Index(fold, index_train, index_test) for fold, (index_train, index_test) in enumerate(list_index.split(x, y))])

In [22]:
piperaceae_dataset = Dataset(cfg, os.path.join(cfg["path_base"], "grayscale/images_grayscale_400_400"), os.path.join(cfg["path_base"], "mask"))
len(getattr(piperaceae_dataset, "list_images"))

375

In [23]:
list_images = getattr(piperaceae_dataset, "list_images")
x = numpy.array(list([l for l in list_images]), dtype=object)
y = numpy.array(list([getattr(l["image"], "label").value for l in list_images]))

collections.Counter(y.tolist())

Counter({'manekia': 75,
         'ottonia': 75,
         'peperomia': 75,
         'piper': 75,
         'pothomorphe': 75})

In [24]:
list_index = get_list_index(cfg, x, y)
len(list_index)

2

In [ ]:
from metrics import jaccard_distance, dice_coef


def get_image(cfg, list_images):
    x = list([])
    y = list([])
    for image in list_images:
        x.append(getattr(image["image"], "image"))
        y.append(getattr(image["mask"], "image"))
    x = numpy.array(x)
    y = numpy.array(y)
    return x.reshape(x.shape[0], cfg["image_size"], cfg["image_size"], 1), y.reshape(y.shape[0], cfg["image_size"],
                                                                                     cfg["image_size"], 1)


list_result = list([])
list_elapsed_time = list([])
path = get_path_mean(cfg)
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
for index in list_index:
    fold = str(getattr(index, "fold"))
    print(fold)
    path_outfile_each_fold = get_path_outfile_each_fold(fold, path)
    pathlib.Path(path_outfile_each_fold).mkdir(parents=True, exist_ok=True)

    x_train_o, y_train_o = x[getattr(index, "index_train")], y[getattr(index, "index_train")]
    x_test_o, y_test_o = x[getattr(index, "index_test")], y[getattr(index, "index_test")]
    x_train_o, x_val_o, y_train_o, y_val_o = sklearn.model_selection.train_test_split(x_train_o, y_train_o,
                                                                                      test_size=cfg["val_size"],
                                                                                      random_state=cfg["random_state"])

    x_train, y_train = get_image(cfg, x_train_o)
    x_test, y_test = get_image(cfg, x_test_o)
    x_val, y_val = get_image(cfg, x_val_o)

    print(x_train.shape, y_train.shape)
    print(x_test.shape, y_test.shape)
    print(x_val.shape, y_val.shape)

    # unet_filename = "out/31-07-2022-18-09-15/0/batch8+lr0_001+epoch3+steps36+fold0+unet.h5"

    checkpointer, reduce_learning_rate, steps_per_epoch, strategy, train_generator, unet_filename = cfg_model(cfg, fold, path_outfile_each_fold, x_train, y_train)
    elapsed_time = None
    if os.path.exists(unet_filename):
        model = tensorflow.keras.models.load_model(unet_filename, custom_objects={  "jaccard_distance_loss": jaccard_distance_loss,
        "dice_coef": dice_coef, "jaccard_distance": jaccard_distance
    })
    else:
        elapsed_time, model = train_model(cfg, checkpointer, fold, path_outfile_each_fold, reduce_learning_rate, steps_per_epoch, strategy, train_generator, x_val, y_val)

    metrics = calculate_iou_dice(model, x_test, x_train, x_val, y_test, y_train, y_val)
    list_elapsed_time.append(elapsed_time)
    list_result.append(metrics)
    create_outfile_each_fold(elapsed_time, fold, metrics, path_outfile_each_fold)
    save_figs(cfg, model, path_outfile_each_fold, x_test_o, x_train_o, x_val_o, y_test_o, y_train_o, y_val_o)

0
(285, 400, 400, 1) (285, 400, 400, 1)
(75, 400, 400, 1) (75, 400, 400, 1)
(15, 400, 400, 1) (15, 400, 400, 1)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
36/36 [==============================] - ETA: 0s - loss: 0.6815 - dice_coef: 0.4753 - jaccard_distance: 0.3201
Epoch 1: val_loss improved from inf to 0.87404, saving model to out\31-07-2022-20-50-59\0\batch8+lr0_001+epoch1+steps36+fold0+unet.h5
36/36 [==============================] - 124s 3s/step - loss: 0.6815 - dice_coef: 0.4753 - jaccard_distance: 0.3201 - val_loss: 0.8740 - val_dice_coef: 0.2237 - val_jaccard_distance: 0.1260 - lr: 0.0010
time elapsed 00:02:05
out\31-07-2022-20-50-59\0\fold0-fit.pckl created
out\31-07-2022-20-50-59\0\fold0-lossgraph.png created
['loss', 'dice_coef', 'jaccard_distance']
fold: 0, elapsed_time: 00:02:05


C:\Users\xandao\Documents\GitKraken\piperaceae-segmentation\files.py:110: UserWarning: out\31-07-2022-20-50-59\0\test\0\228piperALCB002452+img+mask_unet.png is a low contrast image
  skimage.io.imsave(filename, skimage.img_as_ubyte(image))
C:\Users\xandao\Documents\GitKraken\piperaceae-segmentation\files.py:110: UserWarning: out\31-07-2022-20-50-59\0\test\2\70manekiaV0489288F_01+img+mask_unet.png is a low contrast image
  skimage.io.imsave(filename, skimage.img_as_ubyte(image))
C:\Users\xandao\Documents\GitKraken\piperaceae-segmentation\files.py:110: UserWarning: out\31-07-2022-20-50-59\0\test\3\37manekiaNY01801087_01+img+mask_unet.png is a low contrast image
  skimage.io.imsave(filename, skimage.img_as_ubyte(image))
C:\Users\xandao\Documents\GitKraken\piperaceae-segmentation\files.py:110: UserWarning: out\31-07-2022-20-50-59\0\test\5\138ottoniaUEC146277+img+mask_unet.png is a low contrast image
  skimage.io.imsave(filename, skimage.img_as_ubyte(image))
C:\Users\xandao\Documents\GitKra

In [ ]:
create_outfile_mean(cfg, list_elapsed_time, list_result, path)
len(list_result)